Redes Sociais: [GitHub](https://github.com/wendelfrota) | [Linkedin](www.linkedin.com/in/wendel-frota-11649b279)

## Bilioteca do HuggingFace para Conjuntos de Dados:

In [ ]:
pip install datasets


## Importação das Bibliotecas

In [ ]:
import numpy as np
from datasets import load_dataset
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

## Declaração dos Hiperparâmetros e Variáveis

Explicarei cada uma delas mais à frente

In [ ]:
# Hyperparameters
vocab = 12000
embedding_dim = 60
max_length = 220
optimizer = 'adam'
loss = 'binary_crossentropy'
metrics = ['accuracy', 'mae']
batch = 32
epochs = 9

## Separando o Conjunto de Dados

Nessa célula estamos:

* Carregando o conjunto de dados do imdb, que possui:
> Avaliações\
> Rótulos (1-Boa, 2-Ruim)
* Separando o conjunto em:
> Avaliações e Rótulos de TREINO\
> Avaliações e Rótulos de TESTE

\

**Importante ressaltar que à todo momento iremos utilizar a biblioteca NumPy, pois:**
* É mais veloz que as listas convencionais do Python
* Realiza uma ampla variedade de operações matemáticas em matrizes
* Fornece uma enorme biblioteca de funções matemáticas de alto nível

In [ ]:
# Dataset
dataset = load_dataset('imdb')
train_reviews = np.array(dataset['train']['text'])
train_labels = np.array(dataset['train']['label'])

test_reviews = np.array(dataset['test']['text'])
test_labels = np.array(dataset['test']['label'])

## Preparando os Dados

Nessa célula estamos:

* Inicializando um objeto Tokenizer, que faz a transformação de texto em sequências de tokens, com:
> * **num_words=vocab:** Um tamanho de vocabulário (variável definida no inicio do notebook)
> * **oov_token=<OOV\>:** Se uma palavra não estiver no vocabulário, ela será substituída pelo token <OOV> (Out Of Vocabulary), útil para palavras fora do vocabulário que aparecem durante a fase de teste ou inferência.

* Ajustando o Tokenizer aos textos de treinamento para construir o índice de palavras, ou seja, estamos criando uma representação numérica correspondente

* Convertendo as avaliações de treinamento e de teste em sequências de números inteiros

* Preenchendo as sequências de treinamento e teste para terem o mesmo comprimento, com:
> * **maxlen=max_length:** Comprimento máximo das sequências, truncando/removendo ou preenchendo sequências não adequadas (variável definida no inicio do notebook)
> * **truncating='post':** Define que o truncamento deve ser feito no final da sequência
> * **padding='post':** Define que o preenchimento deve ser feito no final da sequência

Além de criar uma tupla para validação do modelo mais à frente

In [ ]:
# Tokenizer
tokenizer = Tokenizer(num_words=vocab, oov_token='<OOV>')
tokenizer.fit_on_texts(train_reviews)
train_reviews = tokenizer.texts_to_sequences(train_reviews)
test_reviews = tokenizer.texts_to_sequences(test_reviews)
train_reviews = pad_sequences(train_reviews, maxlen=max_length, truncating='post', padding='post')
test_reviews = pad_sequences(test_reviews, maxlen=max_length, truncating='post', padding='post')

val_data = (test_reviews, test_labels)

## Criando Modelo

E, finalmente, estamos criando as redes neurais do nosso modelo:
* Instanciando um modelo sequencial
* Adicionando camadas de embedding (variáveis definidas no inicio do notebook):
> **Embedding:** Resumidamente, a camada aprende representações densas e contínuas das palavras, capturando semelhanças semânticas e contextuais
> * **input_dim=vocab:** Tamanho do vocabulário
> * **output_dim=embedding_dim:** Tamanho da dimensão do vetor de embedding, ou seja, define qual o tamanho do vetor que representará cada palavra.
> * **input_length=max_length:** Comprimento máximo da sequência de input.
* Adicionando camadas de LSTM (Long Short-Term Memory):
> **LSTM:** É projetado para lidar com informações de longo prazo
* Adicionando cama dense:
> **Dense:** É uma camada totalmente conectada à camada anterior e produz uma única saída neste caso (classificação binária)
> * **activation='sigmoid':** Função de ativação para converter a saída da rede em uma probabilidade

In [ ]:
# Model
model = Sequential()

# Model - Layers
model.add(Embedding(input_dim=vocab, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(60))
model.add(Dense(1, activation='sigmoid'))

## Treinando Modelo

Agora, na reta final do modelo:
* Configuramos o modelo (variáveis definidas no inicio do notebook):
> * **optimizer=optimizer:**  Algoritmo para ajustar os pesos da rede neural durante o treinamento e minimizar a função de perda\
Estamos usando o Adam (Adaptive Moment Estimation)
> * **loss=loss:** Função de perda que queremos otimizar\
Estamos usando o binary_crossentropy para classificação binária
> * **metrics=metrics:** Métricas para avaliar o desempenho do modelo durante o treinamento\
Estamos usando o MAE (Mean Absolute Error) e accuracy
* Treinamos o modelo (variáveis definidas no inicio do notebook):
> * **batch_size=batch:** O treinamento geralmente é feito em lotes para que o modelo não precise processar todo o conjunto de dados de uma só vez
> * **epochs=epochs:** Número de vezes que o modelo passará por todo o conjunto de dados durante o treinamento
> * **validation_data=val_data:** São usados para avaliar o desempenho do modelo, sendo útil para detectar problemas de sobreajuste (overfitting)\
Os resultados da validação não são usados para ajustar os pesos do modelo
* Verificamos as informações do modelo já treinado
* Salvamos

In [ ]:
# Model - Fit
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
result = model.fit(train_reviews, train_labels, batch_size=batch, epochs=epochs, validation_data=val_data)


# Model - Info & Save
model.summary()
model.save('./model_trained_v1.keras')

## Salvando Histórico de Treinamento

In [ ]:
# Log
with open('history_log.txt', 'w') as f:
    for key, value in result.history.items():
        f.write(f"{key}: {value}\n")

## Imprimindo Gráficos de Treinamento

In [ ]:
# Matplotlib
plt.plot(result.history['loss'], label='Erro treino')
plt.plot(result.history['val_loss'], label='Erro teste')
plt.title('Histórico de Treinamento - Função de custo')
plt.ylabel('Função de custo')
plt.xlabel('Épocas de treinamento')
plt.legend()
plt.show()

plt.plot(result.history['accuracy'], label='Acurácia treino')
plt.plot(result.history['val_accuracy'], label='Acurácia teste')
plt.title('Histórico de Treinamento - Acurácia')
plt.ylabel('Acurácia')
plt.xlabel('Épocas de treinamento')
plt.legend()
plt.show()

## Quer aprender mais um pouquinho das tecnologias utilizadas aqui? Confira:

### NumPy:
* Site oficial: [numpy.org](https://numpy.org)
* Qual a funcionalidade do NumPy? + Características [ [Clique aqui](https://cursos.alura.com.br/forum/topico-qual-a-funcionalidade-do-numpy-128044#:~:text=NumPy%20tem%20como%20objetivo%20fornecer,as%20listas%20tradicionais%20do%20Python.) ]
* Por que o NumPy é mais rápido em Python? [ [Clique aqui](https://acervolima.com/por-que-o-numpy-e-mais-rapido-em-python/) ]


### Keras
* Site oficial: [keras.io](https://keras.io)
* O que é Keras e para que serve? [ [Clique aqui](https://didatica.tech/o-que-e-keras-para-que-serve/) ]
* Keras: The high-level API for TensorFlow [ [Clique aqui](https://www.tensorflow.org/guide/keras) ]

### Matplotlib
* Site oficial: [matplotlib.org](https://matplotlib.org)
* Matplotlib uma biblioteca Python para gerar gráficos interessantes [ [Clique aqui](https://www.alura.com.br/artigos/criando-graficos-no-python-com-a-matplotlib) ]

### HuggingFace
* Site oficial: [huggingface.co](https://huggingface.co)